# A Guided Tour of LSST Calexps
<br>Owner(s): **David Shupe** ([@stargaser](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@stargaser))
<br>Last Verified to Run: **2019-08-13**
<br>Verified Stack Release: **v18.1.0**

We'll inspect a visit image ``calexp`` object, and then show how a coadd image differs.

### Learning Objectives:

After working through this tutorial you should be able to follow some best practices when working with LSST ``calexp`` (image) objects.

### Logistics
This notebook is intended to be runnable on `lsst-lsp-stable.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.

## Set-up

In [ ]:
from lsst.daf.persistence import Butler

In [ ]:
import lsst.afw.display as afw_display

### Retrieving and inspecting a calexp

For the first part of this tutorial we will use simulated LSST data from Twinkles, see https://github.com/LSSTDESC/Twinkles/blob/master/README.md

Define a data directory and create a Butler

In [ ]:
datadir = '/project/shared/data/Twinkles_subset/output_data_v2'
butler = Butler(datadir)

Other notebooks show how to view what data are available in a Butler object. Here we get a specific one.

In [ ]:
dataId = {'filter': 'r', 'raft': '2,2', 'sensor': '1,1', 'visit': 235}
calexp = butler.get('calexp', **dataId)

### Image planes / pixel data

In terms of pixel data, a calexp contains an image, a mask, and a variance.

Let's see how to access the image.

In [ ]:
calexp.image

To show the pixel data, we will make use of the matplotlib backend to `lsst.afw.display`.

Due to current limitations of this backend, the display must be defined and used in the same code cell, much as matplotlib commands in a notebook must all be in one cell to produce a plot.

In [ ]:
%matplotlib inline

If the entire calexp is displayed, masks will be overlaid. Here we will eschew the mask display by showing only the image.

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale("asinh", "zscale")
display1.mtv(calexp.image)

To access the pixel values as an array, use the `.array` attribute.

In [ ]:
data = calexp.image.array
data

In [ ]:
data.__class__

Let's list all the methods for our calexp.

In [ ]:
calexp_methods = [m for m in dir(calexp) if not m.startswith('_')]

In [ ]:
calexp_methods

Access the masked Image

In [ ]:
calexp.maskedImage

Access the variance object and the underlying Numpy array

In [ ]:
calexp.variance

In [ ]:
calexp.variance.array

Access the mask and its underlying array

In [ ]:
calexp.mask

In [ ]:
calexp.mask.array

Get the dimensions of the image, mask and variance

In [ ]:
calexp.getDimensions()

The image, maskedImage and Exposure objects in `lsst.afw.display` include information on **LSST pixels**, which are 0-based with an optional offset.

For a calexp these are usually zero.

In [ ]:
calexp.getXY0()

In [ ]:
calexp.getX0(), calexp.getY0()

### Metadata

Access the wcs object

In [ ]:
wcs = calexp.getWcs()
wcs

The WCS object can be used e.g. to convert pixel coordinates into sky coordinates

In [ ]:
wcs.pixelToSky(100.0, 100.0)

Let's try accessing the metadata, and see what (header) keywords we have.

In [ ]:
metadata = calexp.getMetadata()
# help(metadata)
print(metadata.getOrderedNames())

In [ ]:
metadata.get('CCDTEMP')

### Better metadata: ExposureInfo and VisitInfo

For many purposes, information about an exposure is obtainable via the ExposureInfo and VisitInfo classes.

In [ ]:
calexp_info = calexp.getInfo()

In [ ]:
visit_info = calexp_info.getVisitInfo()

In [ ]:
dir(visit_info)

Obtain weather information for this visit

In [ ]:
visit_info.getWeather()

Check if this calexp has a valid polygon

In [ ]:
calexp_info.hasValidPolygon()

Since there is no valid polygon, the `polygon` variable in the next cell gets the value of None.

In [ ]:
polygon = calexp_info.getValidPolygon()

The calexp is not a coadd so this method returns False.

In [ ]:
calexp_info.hasCoaddInputs()

Does the calexp contain transmission curve information?

In [ ]:
calexp_info.hasTransmissionCurve()

Does the calexp contain a World Coordinate System?

In [ ]:
calexp_info.hasWcs()

Does the calexp have a detector?

In [ ]:
calexp_info.hasDetector()

In [ ]:
dir(calexp_info.getDetector())

Does the calexp have an aperture correction map?

In [ ]:
calexp_info.hasApCorrMap()

Let's get the aperture correctin map and print some information about it

In [ ]:
apCorrMap = calexp_info.getApCorrMap()

In [ ]:
for k in apCorrMap.keys():
    print(k, apCorrMap.get(k))

### Image PSF

Check if our calexp has a PSF

In [ ]:
calexp.hasPsf()

In [ ]:
psf = calexp.getPsf()

The PSF object can be used to get a realization of a PSF at a specific point

In [ ]:
from lsst.geom import Point2D
psfimage = psf.computeImage(Point2D(100.,100.))

Visualize the PSF

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale('asinh', min=0.0, max=1.e-3, unit='absolute')
display1.mtv(psfimage)

Access the calibration object which can be used to convert instrumental magnitudes to AB magnitudes

In [ ]:
calib = calexp.getCalib()
calib

### Image cutouts

We can make a cutout from the calexp in our session.

In [ ]:
import lsst.afw.geom as afwGeom
import lsst.afw.image as afwImage

In [ ]:
bbox = afwGeom.Box2I()
bbox.include(afwGeom.Point2I(2200,3200))
bbox.include(afwGeom.Point2I(2800,3800))
cutout = calexp[bbox]

Notice that when the image is displayed, the pixel values relate to the parent image.

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale('asinh', 'zscale')
display1.mtv(cutout.image)

The coordinate of the lower-left-hand pixel is XY0.

In [ ]:
cutout.getXY0()

If a cutout was all that was desired from the start, we could have used our BoundingBox together with our Butler to have read in only the cutout.

In [ ]:
cutout_calexp = butler.get('calexp_sub', bbox=bbox, dataId=dataId)
cutout_calexp.getDimensions()

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale('asinh', 'zscale')
display1.mtv(cutout_calexp.image)

The `clone` method makes a deep copy. The result can be sliced with a BoundingBox

In [ ]:
clone_cutout = calexp.clone()[bbox]

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale("asinh", "zscale")
display1.mtv(clone_cutout.image)

## Repeat for a coadd

For this last section, we will use Hyper Suprime-Cam (HSC) data that has been modified for tutorial purposes.

In [ ]:
coadd_butler = Butler('/project/shared/data/with-globular/')

In [ ]:
coadd_butler.getKeys('deepCoadd_calexp')

We cannot use queryMetadata to look up what is available for coadds. This will be fixed in Butler Gen3.

For the time being, open a terminal and list files in `/project/shared/data/with-globular/` to see what's available. Or just carry on using the following example:

In [ ]:
dataId = {'filter':'HSC-I', 'tract':9813, 'patch':'4,4'}

Retrieve a coadd `calexp`, and see what methods it provides.

In [ ]:
coadd = coadd_butler.get('deepCoadd_calexp', dataId)

In [ ]:
coadd_methods = [m for m in dir(coadd) if not m.startswith('_')]

In [ ]:
coadd_methods

In [ ]:
set(coadd_methods).symmetric_difference(set(calexp_methods))

The result of the `set` command above shows that a calexp and a coadd have the same methods. This is expected, because they are the same class.

In [ ]:
print(calexp.__class__, coadd.__class__)

A `deepCoadd_calexp` and a visit `calexp` differ mainly in the masks and the xy0 value.

In [ ]:
calexp.mask.getMaskPlaneDict()

In [ ]:
coadd.mask.getMaskPlaneDict()

In [ ]:
coadd.getXY0()

Display the coadd with all masks visible.

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale("asinh", "zscale")
display1.mtv(coadd)

Display the image data only with a zoom and pan to some nice-looking galaxies, to show off our hyperbolic arcsine stretch:

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale("asinh", "zscale")
display1.mtv(coadd.image)
display1.zoom(16)
display1.pan(18700, 17000)